## Introduction

<p>One useful data point in detecting fraud is the account history of a customer.
For an account, we receive notification of purchases and sometimes reports of fraud. 
Typically, a prior report of fraud for an account would increase the perceied risk of fraud on future transactions.</p>

<p>Similarly, a history of non-fraudulent purchases for an account would decrease the risk of fraud.
A credit card holder has 90 days to report any fraudulent transactions with the card.
So, if an account has purchases over 90 days old and no reports of fraud, we assume that these older purchases were not fraudulent.</p>


## Describing the Problem

<p>The purpose of this programming problem is to determine the status of a customer account history at the time a new purchase is made.</p>

<p>The input is a sequence of customer account events, in chronological order. Each event has three fields, all of which are of string type</p> 

<code>&lt;DATE&gt;, &lt;CUSTOMER_ACCOUNT_ID&gt;, &lt;EVENT_TYPE&gt;</code>
    

<p>For example:</p>

<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-01,fraudster@fraud.com,FRAUD_REPORT</code></li>
</ul>

<div>
<p>There are two event types</p>
    <ol>
        <li><code>PURCASE</code> - indicates a purchase by this customer account on the specified date.</li>
        <li><code>FRAUD_REPORT</code> - indicates we received a report of fraud associated with this customer account. The specified date is that date that we received the report, not the date the fraud was committed.</li>
    </ol>
</div>

<p>For each event <code>PURCHASE</code>, we are interested in a summary of the customer account history based on prior events. The summary consists of the date of the summary, the customer account ID, and a status.</p>
<br />
<div>
<p>There are four possible values for the status of the customer account history:</p>
    <ol>
        <li><code>NO_HISTORY</code> - there are no prior events for this customer account</li>
        <li><code>FRAUD_HISTORY</code> - we have at least one event <code>FRAUD_REPORT</code> for this customer account</li>
        <li><code>GOOD_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior that <code>PURCHASE</code> is more than 90 days old</li>
        <li><code>UNCONFIRMED_HISTORY</code> - customer account has no <code>FRAUD_REPORT</code> and at least one prior <code>PURCHASE</code> but no <code>PURCHASE</code> over 90 days old</li>
    </ol>
</div>

<p>For accounts with <code>FRAUD_HISTORY</code>, <code>GOOD_HISTORY</code>, and <code>UNCONFIRMED_HISTORY</code>, the status also contains a count of relevant events.</p>
<ul style="list-style-type:none;">
    <li><code>FRAUD_HISTORY</code> - count of <code>FRAUD_REPORT</code></li>
    <li><code>GOOD_HISTORY</code> - count of prior <code>PURCHASE</code> over 90 days old</li>
    <li><code>UNCONFIRMED_HISTORY</code> - count of prior <code>PURCHASE</code></li>
</ul>
<p>The output is expected to be in the same order as the input</p>

## Sample Input and Output

<p>For the following input:</p>

<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-01,fraudster@fraud.com,FRAUD_REPORT</code></li>
    <li><code>2015-02-03,fraudster@fraud.com,FRAUD_REPORT</code></li>
    <li><code>2015-02-10,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-02-14,fraudster@fraud.com,PURCHASE</code></li>
    <li><code>2015-03-15,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-05-01,joe@signifyd.com,PURCHASE</code></li>
    <li><code>2015-10-01,joe@signifyd.com,PURCHASE</code></li>
</ul>
<br />
<ul style="list-style-type:none;">
    <li><code>2015-01-01,joe@signifyd.com,NO_HISTORY</code></li>
    <li><code>2015-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1</code></li>
    <li><code>2015-02-14,fraudster@fraud.com,FRAUD_HISTORY:2</code></li>
    <li><code>2015-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2</code></li>
    <li><code>2015-05-01,joe@signifyd.com,GOOD_HISTORY:1</code></li>
    <li><code>2015-10-01,joe@signifyd.com,GOOD_HISTORY:4</code></li>
</ul>

In [2]:
from csv import reader

# open file in read mode

with open('input.csv', 'r') as read_obj:
    # pass file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    
    # iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        print(row)

['2015-01-01', 'joe@signifyd.com', 'PURCHASE']
['2015-02-01', 'fraudster@fraud.com', 'FRAUD_REPORT']
['2015-02-03', 'fraudster@fraud.com', 'FRAUD_REPORT']
['2015-02-10', 'joe@signifyd.com', 'PURCHASE']
['2015-02-14', 'fraudster@fraud.com', 'PURCHASE']
['2015-03-15', 'joe@signifyd.com', 'PURCHASE']
['2015-05-01', 'joe@signifyd.com', 'PURCHASE']
['2015-10-01', 'joe@signifyd.com', 'PURCHASE']
